In [ ]:
import SAGA
from easyquery import Query, QueryMaker
from SAGA.database import FitsTable
from SAGA import ObjectCuts as C
from SAGA import HostCuts as H

from itertools import combinations
from collections import Counter, defaultdict
from astropy.table import Table, join, vstack
from astropy.coordinates import SkyCoord, Distance
from SAGA.utils import add_skycoord
from easyquery import Query, QueryMaker
from astropy.io import ascii

import SAGA
from SAGA.database import FitsTable, GoogleSheets
from SAGA import ObjectCuts as C
from SAGA.utils import add_skycoord, fill_values_by_query
from SAGA.utils.distance import z2v, d2m, m2d, v2z
from SAGA.utils.display import show_images
from SAGA.objects.object_catalog import calc_fiducial_p_sat, calc_fiducial_p_sat_corrected
from scipy.stats import spearmanr
print(SAGA.__version__)

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
hostlist = saga.host_catalog.load(include_stats=True)

In [ ]:
def annotate_base(base):
    base["Mr_"] = base["r_mag"] - d2m(base["HOST_DIST"])
    base["Mr"] = np.where(np.isfinite(base["Mr"]) & C.has_spec.mask(base), base["Mr"], base["Mr_"])
    base["log_sm"] = np.where(np.isfinite(base["log_sm"]), base["log_sm"], 1.254 + 1.0976 * base["gr"] - 0.4 * base["Mr"])
    
    base["human_selected"] = 0
    hs = saga.database["human_selected"].read()
    for i in range(1, 4):
        base["human_selected"] += np.in1d(base["OBJID"], Query(f"score == {i}").filter(hs, "OBJID")).astype(np.int) * i

    return base

In [ ]:
base = SAGA.database.FitsTable("/home/yymao/Documents/Research/SAGA/PaperII/data-archive/saga_base_all.fits").read()
base = saga.host_catalog.construct_host_query("paper2_complete").filter(base)
base = annotate_base(base)

In [ ]:
v2z(H.good_hosts.filter(hostlist, "V_HELIO")-275).min()

In [ ]:
v2z(H.good_hosts.filter(hostlist, "V_HELIO")+275).max()

In [ ]:
_known_telnames = {
    "HOST",
    "2dF",
    "6dF",
    "SDSS",
    "NSA",
    "GAMA",
    "OzDES",
    "2dFLen",
    "WIGGZ",
    "UKST",
    "LCRS",
    "slack",
}
has_our_specs_only = QueryMaker.vectorize(
    lambda x: x and set(x.split("+")).isdisjoint(_known_telnames), "SPEC_REPEAT"
)

In [ ]:
Query(C.has_spec, QueryMaker.contains("SPEC_REPEAT", "ALFALF")).count(base)

In [ ]:
Query(C.has_spec, has_our_specs_only, QueryMaker.contains("SPEC_REPEAT", "ALFALF")).filter(base)

In [ ]:
Query(C.is_sat, "EW_Halpha < 0").count(base)

In [ ]:
from astropy.modeling.functional_models import Sersic1D

In [ ]:
np.log10(Sersic1D.evaluate(0.0000001, 1, 1, 0.2)/Sersic1D.evaluate(, 1, 1, 0.2))

In [ ]:
24 + np.log10(Sersic1D.evaluate(0.0000001, 1, 1, 1)/Sersic1D.evaluate(1, 1, 1, 1))

In [ ]:
24 + np.log10(Sersic1D.evaluate(0.0000001, 1, 1, 0.2)/Sersic1D.evaluate(1, 1, 1, 0.2))